# 🛰️ SNUNet-CBAM Training on LEVIR-CD

## ⚠️ SETUP REQUIRED:
1. **Enable GPU**: Settings → Accelerator → **GPU T4 x2**
2. **Enable Internet**: Settings → Internet → **On**
3. Then run all cells

---

In [ ]:
# Check GPU
import torch
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    raise RuntimeError("GPU NOT ENABLED! Go to Settings -> Accelerator -> GPU T4")

In [ ]:
# Download LEVIR-CD dataset
!pip install -q gdown
import gdown
import zipfile
import os

DATA_DIR = '/kaggle/working/LEVIR-CD'
if not os.path.exists(DATA_DIR):
    print("Downloading LEVIR-CD...")
    # LEVIR-CD 256x256 patches
    gdown.download('https://drive.google.com/uc?id=1786BQHZyPalR38ZVWL4Y0Rl8tD-Y1pwC', 'levir.zip', quiet=False)
    with zipfile.ZipFile('levir.zip', 'r') as z:
        z.extractall('/kaggle/working')
    os.remove('levir.zip')
    print("✅ Done!")
else:
    print("✅ Dataset exists")

In [ ]:
# Find data
import os
for root, dirs, files in os.walk('/kaggle/working'):
    if 'A' in dirs and 'B' in dirs:
        DATA_ROOT = os.path.dirname(root)
        break
print(f"Data: {DATA_ROOT}")

# Create lists
for split in ['train', 'val', 'test']:
    p = os.path.join(DATA_ROOT, split, 'A')
    if os.path.exists(p):
        files = sorted([f for f in os.listdir(p) if f.endswith('.png')])
        with open(f'/kaggle/working/{split}.txt', 'w') as out:
            for f in files:
                out.write(f"{split}/A/{f} {split}/B/{f} {split}/label/{f}\n")
        print(f"{split}: {len(files)}")

In [ ]:
# Model
import torch
import torch.nn as nn
import torch.nn.functional as F

class CBAM(nn.Module):
    def __init__(self, c):
        super().__init__()
        self.ca = nn.Sequential(nn.AdaptiveAvgPool2d(1), nn.Conv2d(c, c//16, 1), nn.ReLU(), nn.Conv2d(c//16, c, 1), nn.Sigmoid())
        self.sa = nn.Sequential(nn.Conv2d(2, 1, 7, padding=3), nn.Sigmoid())
    def forward(self, x):
        x = x * self.ca(x)
        return x * self.sa(torch.cat([x.mean(1,keepdim=True), x.max(1,keepdim=True)[0]], 1))

class ConvBlock(nn.Module):
    def __init__(self, i, o):
        super().__init__()
        self.c = nn.Sequential(nn.Conv2d(i,o,3,padding=1), nn.BatchNorm2d(o), nn.ReLU(), nn.Conv2d(o,o,3,padding=1), nn.BatchNorm2d(o), nn.ReLU())
    def forward(self, x): return self.c(x)

class SNUNet(nn.Module):
    def __init__(self, c=32):
        super().__init__()
        self.e1,self.e2,self.e3,self.e4,self.e5 = ConvBlock(3,c), ConvBlock(c,c*2), ConvBlock(c*2,c*4), ConvBlock(c*4,c*8), ConvBlock(c*8,c*16)
        self.pool = nn.MaxPool2d(2)
        self.u4,self.d4,self.a4 = nn.ConvTranspose2d(c*16,c*8,2,2), ConvBlock(c*24,c*8), CBAM(c*8)
        self.u3,self.d3,self.a3 = nn.ConvTranspose2d(c*8,c*4,2,2), ConvBlock(c*12,c*4), CBAM(c*4)
        self.u2,self.d2,self.a2 = nn.ConvTranspose2d(c*4,c*2,2,2), ConvBlock(c*6,c*2), CBAM(c*2)
        self.u1,self.d1,self.a1 = nn.ConvTranspose2d(c*2,c,2,2), ConvBlock(c*3,c), CBAM(c)
        self.out = nn.Conv2d(c,1,1)

    def forward(self, x1, x2):
        e11,e12 = self.e1(x1), self.e1(x2)
        e21,e22 = self.e2(self.pool(e11)), self.e2(self.pool(e12))
        e31,e32 = self.e3(self.pool(e21)), self.e3(self.pool(e22))
        e41,e42 = self.e4(self.pool(e31)), self.e4(self.pool(e32))
        e51,e52 = self.e5(self.pool(e41)), self.e5(self.pool(e42))
        d = torch.abs(e51-e52)
        d = self.a4(self.d4(torch.cat([self.u4(d),e41,e42],1)))
        d = self.a3(self.d3(torch.cat([self.u3(d),e31,e32],1)))
        d = self.a2(self.d2(torch.cat([self.u2(d),e21,e22],1)))
        d = self.a1(self.d1(torch.cat([self.u1(d),e11,e12],1)))
        return self.out(d)

print(f"Params: {sum(p.numel() for p in SNUNet().parameters()):,}")

In [ ]:
# Dataset
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np

class CDData(Dataset):
    def __init__(self, root, lst):
        self.root = root
        with open(lst) as f: self.items = [l.strip().split() for l in f]
    def __len__(self): return len(self.items)
    def __getitem__(self, i):
        a,b,l = self.items[i]
        im1 = torch.from_numpy(np.array(Image.open(f"{self.root}/{a}").resize((256,256)))).permute(2,0,1).float()/255
        im2 = torch.from_numpy(np.array(Image.open(f"{self.root}/{b}").resize((256,256)))).permute(2,0,1).float()/255
        lb = (torch.from_numpy(np.array(Image.open(f"{self.root}/{l}").convert('L').resize((256,256)))).float()/255>0.5).float().unsqueeze(0)
        return im1, im2, lb

train_ds = CDData(DATA_ROOT, '/kaggle/working/train.txt')
val_ds = CDData(DATA_ROOT, '/kaggle/working/val.txt')
print(f"Train: {len(train_ds)}, Val: {len(val_ds)}")

In [ ]:
# Training
from tqdm import tqdm
from sklearn.metrics import f1_score
import csv

device = torch.device('cuda')
model = SNUNet(32).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=1e-4)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, 100)

train_ld = DataLoader(train_ds, 8, shuffle=True, num_workers=2, pin_memory=True)
val_ld = DataLoader(val_ds, 16, num_workers=2)

os.makedirs('/kaggle/working/ckpt', exist_ok=True)
log = open('/kaggle/working/ckpt/log.csv','w',newline='')
writer = csv.writer(log)
writer.writerow(['ep','loss','f1'])

best = 0
for ep in range(100):
    model.train()
    for x1,x2,y in tqdm(train_ld, desc=f"Ep{ep+1}", leave=False):
        x1,x2,y = x1.to(device), x2.to(device), y.to(device)
        opt.zero_grad()
        out = model(x1,x2)
        loss = F.binary_cross_entropy_with_logits(out,y) + (1 - (2*(torch.sigmoid(out)*y).sum()+1)/(torch.sigmoid(out).sum()+y.sum()+1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        opt.step()
    
    model.eval()
    f1s = []
    with torch.no_grad():
        for x1,x2,y in val_ld:
            p = (torch.sigmoid(model(x1.to(device),x2.to(device)))>0.5).cpu().numpy().flatten()
            f1s.append(f1_score(y.numpy().flatten(), p, zero_division=0))
    f1 = sum(f1s)/len(f1s)
    sched.step()
    
    print(f"Ep {ep+1}: F1={f1:.4f}")
    writer.writerow([ep+1, loss.item(), f1])
    log.flush()
    
    if f1 > best:
        best = f1
        torch.save(model.state_dict(), f'/kaggle/working/ckpt/best_{f1:.4f}.pth')
        print(f"  Saved!")

log.close()
print(f"\nBest F1: {best:.4f}")

In [ ]:
# Visualize
import matplotlib.pyplot as plt
import glob

ckpt = max(glob.glob('/kaggle/working/ckpt/best_*.pth'))
model.load_state_dict(torch.load(ckpt))
model.eval()

fig, ax = plt.subplots(3,4,figsize=(12,9))
for r in range(3):
    x1,x2,y = val_ds[r*10]
    with torch.no_grad():
        p = (torch.sigmoid(model(x1.unsqueeze(0).cuda(), x2.unsqueeze(0).cuda()))>0.5).cpu().squeeze().numpy()
    ax[r,0].imshow(x1.permute(1,2,0)); ax[r,1].imshow(x2.permute(1,2,0))
    ax[r,2].imshow(y.squeeze(),cmap='gray'); ax[r,3].imshow(p,cmap='gray')
    for c in range(4): ax[r,c].axis('off')
ax[0,0].set_title('T1'); ax[0,1].set_title('T2'); ax[0,2].set_title('GT'); ax[0,3].set_title('Pred')
plt.tight_layout()
plt.savefig('/kaggle/working/results.png')
plt.show()